In [20]:
from parse_deck import parse_events_from_official
from parse_deck import reassign_category
from deck_category_helper import find_category

In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import re
from tqdm import tqdm

import pandas as pd
from collections import OrderedDict

import json
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [22]:
RUN_PARSE = False

In [23]:
# decks = {
#     deck category 1: [
#         {
#             deck_link: "",
#             deck_code: "",
#             pokemons: {},
#             tools: {},
#             supporters: {},
#             stadiums: {},
#             energies: {},
#             rank: 1,
#             num_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# }

decks = {}
store_file_name = 'deck_db/store.json'
if os.path.exists(store_file_name):
    with open(store_file_name, 'r') as f:
        decks = json.load(f)
        
decks = reassign_category(decks)
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])

dict_keys(['レジ', 'ルギアVSTAR', 'オリジンパルキアVSTAR', 'ムゲンダイナVMAX', 'LOST_ギラティナVSTAR', 'キュレムVMAX', 'Other_Lost', 'ルナトーン', 'ミュウVMAX', 'アル_ジュラルドン', 'アル_そらをとぶピカチュウ', 'others', 'レジエレキVMAX', 'LTB_空の封印石_水超闘雷', 'LTB_空の封印石_水超鋼雷', 'LTB_空の封印石_水草超鋼闘雷', 'LTB_空の封印石_水超雷', 'LTB_空の封印石_水超鋼', 'LTB_空の封印石_超鋼雷', 'アルセウス裏工作', 'LTB_カイオーガ', 'ハピナスV', 'LTB', 'ヒスイ ゾロアークVSTAR', 'ヒスイ ダイケンキVSTAR', 'ロトムVSTAR', 'ゾロア', 'プテラVSTAR', 'ガラル マタドガス', 'LTB_ヤミラミ_リザードン', 'Other_ギラティナVSTAR', 'アル_レジドラゴVSTAR', 'ミュウツーV-UNION', 'LTB_空の封印石_other', 'レジドラゴVSTAR', 'オリジンディアルガVSTAR', 'こくばバドレックスVMAX', 'ゾロア_ウインディ'])


In [24]:
len(store_code_list)

926

In [25]:
# parse events and save to json
if RUN_PARSE:
    parse_events_from_official(decks, store_code_list, num_page_in_event=2) #, page_limit=30, event_limit=1000, num_page_in_event=2

with open(store_file_name, 'w') as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [26]:
# show all categories we have
for k in decks.keys():
    print(f"[{k}]:\n{len(decks[k])}\n")

[レジ]:
53

[ルギアVSTAR]:
272

[オリジンパルキアVSTAR]:
20

[ムゲンダイナVMAX]:
36

[LOST_ギラティナVSTAR]:
29

[キュレムVMAX]:
22

[Other_Lost]:
18

[ルナトーン]:
8

[ミュウVMAX]:
127

[アル_ジュラルドン]:
62

[アル_そらをとぶピカチュウ]:
25

[others]:
37

[レジエレキVMAX]:
22

[LTB_空の封印石_水超闘雷]:
16

[LTB_空の封印石_水超鋼雷]:
18

[LTB_空の封印石_水草超鋼闘雷]:
1

[LTB_空の封印石_水超雷]:
4

[LTB_空の封印石_水超鋼]:
1

[LTB_空の封印石_超鋼雷]:
1

[アルセウス裏工作]:
9

[LTB_カイオーガ]:
20

[ハピナスV]:
9

[LTB]:
31

[ヒスイ ゾロアークVSTAR]:
9

[ヒスイ ダイケンキVSTAR]:
4

[ロトムVSTAR]:
1

[ゾロア]:
17

[プテラVSTAR]:
3

[ガラル マタドガス]:
8

[LTB_ヤミラミ_リザードン]:
21

[Other_ギラティナVSTAR]:
2

[アル_レジドラゴVSTAR]:
3

[ミュウツーV-UNION]:
4

[LTB_空の封印石_other]:
3

[レジドラゴVSTAR]:
2

[オリジンディアルガVSTAR]:
4

[こくばバドレックスVMAX]:
2

[ゾロア_ウインディ]:
2



In [27]:
start_date = "2022年12月02日"  # include
end_date = "2022年12月19日"  # include

In [28]:
# show categories we have in the period
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        if deck["date"] >= start_date and deck["date"] <= end_date:
            deck_cnt += 1

    print(f"{k}\t{deck_cnt}\n")

レジ	50

ルギアVSTAR	265

オリジンパルキアVSTAR	19

ムゲンダイナVMAX	33

LOST_ギラティナVSTAR	29

キュレムVMAX	21

Other_Lost	17

ルナトーン	8

ミュウVMAX	126

アル_ジュラルドン	60

アル_そらをとぶピカチュウ	24

others	37

レジエレキVMAX	22

LTB_空の封印石_水超闘雷	16

LTB_空の封印石_水超鋼雷	18

LTB_空の封印石_水草超鋼闘雷	1

LTB_空の封印石_水超雷	4

LTB_空の封印石_水超鋼	1

LTB_空の封印石_超鋼雷	1

アルセウス裏工作	9

LTB_カイオーガ	20

ハピナスV	9

LTB	30

ヒスイ ゾロアークVSTAR	9

ヒスイ ダイケンキVSTAR	4

ロトムVSTAR	1

ゾロア	16

プテラVSTAR	3

ガラル マタドガス	8

LTB_ヤミラミ_リザードン	20

Other_ギラティナVSTAR	2

アル_レジドラゴVSTAR	3

ミュウツーV-UNION	4

LTB_空の封印石_other	3

レジドラゴVSTAR	2

オリジンディアルガVSTAR	4

こくばバドレックスVMAX	2

ゾロア_ウインディ	1



In [29]:
target_category = "LTB_空の封印石_水超鋼雷"
df_list = []
common_cols = ["date", "num_people", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[target_category]):
        deck_link = deck["deck_link"]  # row id
        if deck["date"] < start_date or deck["date"] > end_date:
            continue
        pokecard = OrderedDict()
        pokecard["date"] = deck["date"]
        pokecard["num_people"] = deck["num_people"]
        pokecard["rank"] = deck["rank"]
        pokecard.update(deck[card_type])
        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_link])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_link])])
    df = df.fillna(0)
    
    # sort rows by date
    df = df.sort_values(by=['date'], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols: col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis='rows', numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis='rows')
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks
    
    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows-1, num_rows-2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]
    
    # sort cols by pick rate
    df = df.sort_values('pick_rate', axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols: col_list.remove(c)
    df = df[common_cols + col_list]
    
    # store
    df_list.append(df)

In [30]:
# number of decks for the target category
len(df_list[0]) - 2

18

In [31]:
# prepare folder
csv_folder = f"./csv/{target_category}"
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

In [32]:
df_list[0].to_csv(f"{csv_folder}/{target_category}-pokecard-{start_date}-{end_date}.csv")
df_list[0]

,date,num_people,rank,キュワワー,ウッウ,ヤミラミ,かがやくゲッコウガ,マナフィ,ライコウV,ザマゼンタ,ドラピオンV,ネオラントV,ガラル ジグザグマ,コオリッポ,カイオーガ,ヤレユータン,AR カイオーガ,カビゴン
pick_rate,NaN,NaN,NaN,1.0,1.000000,1.000000,1.0,1.0,1.0,0.944444,0.944444,0.833333,0.333333,0.222222,0.166667,0.111111,0.055556,0.055556
avg_num_used,NaN,NaN,NaN,4.0,1.777778,1.777778,1.0,1.0,1.0,1.882353,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/YcYcc8-l9cfkC-a8c8xx,2022年12月18日(日),32.0,3.0,4.0,1.000000,2.000000,1.0,1.0,1.0,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkFV1F-cfOTK5-1kkdkV,2022年12月18日(日),48.0,5.0,4.0,2.000000,2.000000,1.0,1.0,1.0,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SEyyyX-KjBq17-EXSMyM,2022年12月18日(日),72.0,9.0,4.0,2.000000,1.000000,1.0,1.0,1.0,2.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ypMXRM-IqrRMO-y2pMyU,2022年12月18日(日),64.0,9.0,4.0,2.000000,2.000000,1.0,1.0,1.0,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/UyUM2y-Vb0RVl-MMpUpy,2022年12月18日(日),32.0,3.0,4.0,2.000000,2.000000,1.0,1.0,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ax8Yx8-CL2MUc-ax8c8x,2022年12月18日(日),48.0,5.0,4.0,2.000000,1.000000,1.0,1.0,1.0,2.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/LNNLii-tK2ncE-Hnn9gn,2022年12月18日(日),32.0,5.0,4.0,2.000000,2.000000,1.0,1.0,1.0,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/EyMy22-1KqGgh-3MXyyy,2022年12月18日(日),64.0,9.0,4.0,2.000000,2.000000,1.0,1.0,1.0,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
df_list[1].to_csv(f"{csv_folder}/{target_category}-tool-{start_date}-{end_date}.csv")
df_list[1]

,date,num_people,rank,クイックボール,ミラージュゲート,回収ネット,あなぬけのヒモ,ふうせん,ふつうのつりざお,空の封印石,ヒスイのヘビーボール,バトルVIPパス,いれかえカート,エネルギーリサイクル,エネルギー転送,ともだちてちょう,やまびこホーン,ロストスイーパー
pick_rate,NaN,NaN,NaN,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,0.944444,0.833333,0.833333,0.388889,0.055556,0.055556,0.055556,0.055556
avg_num_used,NaN,NaN,NaN,2.833333,4.0,4.0,2.333333,1.0,1.888889,1.055556,1.000000,3.666667,1.733333,1.285714,1.000000,1.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/YcYcc8-l9cfkC-a8c8xx,2022年12月18日(日),32.0,3.0,3.000000,4.0,4.0,2.000000,1.0,2.000000,1.000000,1.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkFV1F-cfOTK5-1kkdkV,2022年12月18日(日),48.0,5.0,3.000000,4.0,4.0,2.000000,1.0,2.000000,1.000000,1.000000,4.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SEyyyX-KjBq17-EXSMyM,2022年12月18日(日),72.0,9.0,3.000000,4.0,4.0,2.000000,1.0,2.000000,1.000000,1.000000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ypMXRM-IqrRMO-y2pMyU,2022年12月18日(日),64.0,9.0,3.000000,4.0,4.0,3.000000,1.0,2.000000,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/UyUM2y-Vb0RVl-MMpUpy,2022年12月18日(日),32.0,3.0,1.000000,4.0,4.0,2.000000,1.0,2.000000,1.000000,1.000000,4.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ax8Yx8-CL2MUc-ax8c8x,2022年12月18日(日),48.0,5.0,3.000000,4.0,4.0,2.000000,1.0,2.000000,1.000000,1.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/LNNLii-tK2ncE-Hnn9gn,2022年12月18日(日),32.0,5.0,3.000000,4.0,4.0,2.000000,1.0,2.000000,1.000000,1.000000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/EyMy22-1KqGgh-3MXyyy,2022年12月18日(日),64.0,9.0,3.000000,4.0,4.0,4.000000,1.0,1.000000,1.000000,1.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
df_list[2].to_csv(f"{csv_folder}/{target_category}-supporter-{start_date}-{end_date}.csv")
df_list[2]

,date,num_people,rank,アクロマの実験,キバナ,ボスの指令,ツツジ,とりつかい,マリィ,クララ
pick_rate,NaN,NaN,NaN,1.0,1.000000,1.000000,0.333333,0.166667,0.111111,0.055556
avg_num_used,NaN,NaN,NaN,4.0,1.555556,1.444444,1.000000,1.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/YcYcc8-l9cfkC-a8c8xx,2022年12月18日(日),32.0,3.0,4.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkFV1F-cfOTK5-1kkdkV,2022年12月18日(日),48.0,5.0,4.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SEyyyX-KjBq17-EXSMyM,2022年12月18日(日),72.0,9.0,4.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ypMXRM-IqrRMO-y2pMyU,2022年12月18日(日),64.0,9.0,4.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/UyUM2y-Vb0RVl-MMpUpy,2022年12月18日(日),32.0,3.0,4.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ax8Yx8-CL2MUc-ax8c8x,2022年12月18日(日),48.0,5.0,4.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/LNNLii-tK2ncE-Hnn9gn,2022年12月18日(日),32.0,5.0,4.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/EyMy22-1KqGgh-3MXyyy,2022年12月18日(日),64.0,9.0,4.0,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000


In [35]:
df_list[3].to_csv(f"{csv_folder}/{target_category}-stadium-{start_date}-{end_date}.csv")
df_list[3]

,date,num_people,rank,トレーニングコート,シンオウ神殿,ポケストップ
pick_rate,NaN,NaN,NaN,0.722222,0.166667,0.111111
avg_num_used,NaN,NaN,NaN,1.153846,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/YcYcc8-l9cfkC-a8c8xx,2022年12月18日(日),32.0,3.0,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkFV1F-cfOTK5-1kkdkV,2022年12月18日(日),48.0,5.0,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SEyyyX-KjBq17-EXSMyM,2022年12月18日(日),72.0,9.0,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ypMXRM-IqrRMO-y2pMyU,2022年12月18日(日),64.0,9.0,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/UyUM2y-Vb0RVl-MMpUpy,2022年12月18日(日),32.0,3.0,0.000000,0.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ax8Yx8-CL2MUc-ax8c8x,2022年12月18日(日),48.0,5.0,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/LNNLii-tK2ncE-Hnn9gn,2022年12月18日(日),32.0,5.0,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/EyMy22-1KqGgh-3MXyyy,2022年12月18日(日),64.0,9.0,0.000000,0.000000,0.000000


In [36]:
df_list[4].to_csv(f"{csv_folder}/{target_category}-energy-{start_date}-{end_date}.csv")
df_list[4]

,date,num_people,rank,基本鋼エネルギー,基本水エネルギー,基本雷エネルギー,基本超エネルギー,キャプチャーエネルギー
pick_rate,NaN,NaN,NaN,1.000000,1.000000,1.0,1.0,0.888889
avg_num_used,NaN,NaN,NaN,3.777778,3.111111,2.0,2.0,2.250000
https://www.pokemon-card.com/deck/confirm.html/deckID/YcYcc8-l9cfkC-a8c8xx,2022年12月18日(日),32.0,3.0,4.000000,3.000000,2.0,2.0,2.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkFV1F-cfOTK5-1kkdkV,2022年12月18日(日),48.0,5.0,4.000000,3.000000,2.0,2.0,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SEyyyX-KjBq17-EXSMyM,2022年12月18日(日),72.0,9.0,4.000000,4.000000,2.0,2.0,2.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ypMXRM-IqrRMO-y2pMyU,2022年12月18日(日),64.0,9.0,4.000000,3.000000,2.0,2.0,2.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/UyUM2y-Vb0RVl-MMpUpy,2022年12月18日(日),32.0,3.0,2.000000,4.000000,2.0,2.0,3.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ax8Yx8-CL2MUc-ax8c8x,2022年12月18日(日),48.0,5.0,4.000000,2.000000,2.0,2.0,3.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/LNNLii-tK2ncE-Hnn9gn,2022年12月18日(日),32.0,5.0,4.000000,3.000000,2.0,2.0,2.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/EyMy22-1KqGgh-3MXyyy,2022年12月18日(日),64.0,9.0,4.000000,3.000000,2.0,2.0,3.000000


In [37]:
decks["others"]

[{'deck_link': 'https://www.pokemon-card.com/deck/confirm.html/deckID/MypyMy-V9JAzd-pyMypy',
  'deck_code': 'MypyMy-V9JAzd-pyMypy',
  'pokemons': {'アルセウスV': 4,
   'アルセウスVSTAR': 4,
   'スターミーV': 1,
   'ドラピオンV': 1,
   'クロバットV': 1,
   'ネオラントV': 1,
   'かがやくサーナイト': 1},
  'tools': {'クイックボール': 4,
   'ハイパーボール': 4,
   'しんかのおこう': 1,
   '大きなおまもり': 3,
   'あなぬけのヒモ': 1,
   'ポケモンいれかえ': 1,
   'ふつうのつりざお': 1,
   'すごいきずぐすり': 2},
  'supporters': {'メロン': 2,
   'セレナ': 2,
   'マリィ': 3,
   'チェレンの気くばり': 2,
   '博士の研究': 2,
   'ボスの指令': 2},
  'stadiums': {'エイチ湖': 2, '頂への雪道': 1, 'シンオウ神殿': 1, 'ポケストップ': 1},
  'energies': {'基本水エネルギー': 8, 'ダブルターボエネルギー': 4},
  'rank': 9,
  'num_people': 64,
  'date': '2022年12月18日(日)'},
 {'deck_link': 'https://www.pokemon-card.com/deck/confirm.html/deckID/LiL6Hn-gEY8pe-nNgnLL',
  'deck_code': 'LiL6Hn-gEY8pe-nNgnLL',
  'pokemons': {'メッソン': 4,
   'ジメレオン': 4,
   'インテレオン': 1,
   'そらをとぶピカチュウV': 1,
   'そらをとぶピカチュウVMAX': 1,
   'ライコウV': 1,
   'ゼラオラ': 1,
   'かがやくリザードン': 1,
   'カビゴン': 1,
   'マナフィ': 1

In [38]:
len(decks["others"])

37